In [ ]:
# Operational Excellence Analyst DIVE Analysis - E-Commerce Acquisition Target

# ==============================================================================
# SETUP & IMPORTS
# ==============================================================================
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.cloud import bigquery
import numpy as np

# --- BigQuery Configuration ---
# This assumes you have authenticated with Google Cloud in your Colab environment.
# from google.colab import auth
# auth.authenticate_user()

project_id = 'mgmt599-rachakondas-lab1'
dataset_id = 'olist_acquisition_analytics'
client = bigquery.Client(project=project_id)

# ==============================================================================
# DISCOVER PHASE: Delivery Performance & Customer Satisfaction
# ==============================================================================
# Gemini Prompt (for context):
# "I'm the Operational Analyst. My goal is to see if operational issues are
#  causing customer churn. What are the first two things I should analyze?"
# Gemini's Recommendation: Analyze delivery timeliness and customer satisfaction scores.

print("--- Running Discover Phase ---")

# --- Query 1: Delivery Performance and Review Scores ---
sql_discover_ops = f"""
    SELECT
        o.order_id,
        o.order_purchase_timestamp,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date,
        r.review_score
    FROM
        `{project_id}.{dataset_id}.master_table` AS o
    LEFT JOIN
        `{project_id}.{dataset_id}.reviews` AS r ON o.order_id = r.order_id
    WHERE
        o.order_delivered_customer_date IS NOT NULL
        AND o.order_estimated_delivery_date IS NOT NULL;
"""
df_discover_ops = client.query(sql_discover_ops).to_dataframe()

# --- 1. Analyze Delivery Timeliness ---
df_discover_ops['actual_delivery_date'] = pd.to_datetime(df_discover_ops['order_delivered_customer_date'])
df_discover_ops['estimated_delivery_date'] = pd.to_datetime(df_discover_ops['order_estimated_delivery_date'])

# Calculate the difference in days
df_discover_ops['delivery_diff_days'] = (df_discover_ops['actual_delivery_date'] - df_discover_ops['estimated_delivery_date']).dt.days

avg_delay = df_discover_ops['delivery_diff_days'].mean()
on_time_percentage = (df_discover_ops[df_discover_ops['delivery_diff_days'] <= 0].shape[0] / df_discover_ops.shape[0]) * 100

print(f"Average Delivery Delay: {avg_delay:.2f} days")
print(f"Percentage of Orders Delivered On or Before Estimate: {on_time_percentage:.2f}%")

# Visualize the distribution of delivery difference
fig_delivery_hist = px.histogram(
    df_discover_ops,
    x='delivery_diff_days',
    title='Distribution of Delivery Timeliness (Actual vs. Estimated)',
    labels={'delivery_diff_days': 'Days Difference (Actual - Estimated)', 'count': 'Number of Orders'},
    marginal='box'
)
fig_delivery_hist.update_layout(template="plotly_white", title_x=0.5)
fig_delivery_hist.show()


# --- 2. Analyze Customer Satisfaction Scores ---
df_reviews = df_discover_ops.groupby('review_score').size().reset_index(name='review_count')
fig_reviews = px.bar(
    df_reviews,
    x='review_score',
    y='review_count',
    title='Distribution of Customer Review Scores',
    labels={'review_score': 'Review Score (1-5)', 'review_count': 'Number of Reviews'}
)
fig_reviews.update_layout(template="plotly_white", title_x=0.5)
fig_reviews.show()
print("Discovery of operational patterns complete.\n")


# ==============================================================================
# INVESTIGATE PHASE: Identifying the Bottleneck
# ==============================================================================
# Gemini Prompt (for context):
# "Deliveries are consistently late and reviews are polarized. How do I
#  dissect the order timeline to find the root cause of the delays?"
# Gemini's Recommendation: Break the timeline into 'Seller Processing Time' and
# 'Carrier Shipping Time' and compare their distributions.

print("--- Running Investigate Phase: Bottleneck Analysis ---")

# --- Query 2: Full Order Lifecycle Timestamps ---
sql_investigate_ops = f"""
    SELECT
        order_id,
        order_purchase_timestamp,
        order_delivered_carrier_date,
        order_delivered_customer_date
    FROM
        `{project_id}.{dataset_id}.master_table`
    WHERE
        order_delivered_carrier_date IS NOT NULL
        AND order_delivered_customer_date IS NOT NULL;
"""
df_investigate_ops = client.query(sql_investigate_ops).to_dataframe()

# --- Calculate Timeline Stages ---
df_investigate_ops['purchase_ts'] = pd.to_datetime(df_investigate_ops['order_purchase_timestamp'])
df_investigate_ops['carrier_delivery_ts'] = pd.to_datetime(df_investigate_ops['order_delivered_carrier_date'])
df_investigate_ops['customer_delivery_ts'] = pd.to_datetime(df_investigate_ops['order_delivered_customer_date'])

# Calculate the duration of each stage in days
df_investigate_ops['seller_processing_time'] = (df_investigate_ops['carrier_delivery_ts'] - df_investigate_ops['purchase_ts']).dt.total_seconds() / (24 * 60 * 60)
df_investigate_ops['carrier_shipping_time'] = (df_investigate_ops['customer_delivery_ts'] - df_investigate_ops['carrier_delivery_ts']).dt.total_seconds() / (24 * 60 * 60)

# Melt the dataframe for easy plotting with Plotly
df_melted = df_investigate_ops[['seller_processing_time', 'carrier_shipping_time']].melt(var_name='Stage', value_name='Time (Days)')


# --- Visualize the Bottleneck ---
# We filter out extreme outliers for better visualization of the main distribution
q_high = df_melted['Time (Days)'].quantile(0.99)
df_filtered = df_melted[df_melted['Time (Days)'] < q_high]

fig_boxplot = px.box(
    df_filtered,
    x='Stage',
    y='Time (Days)',
    color='Stage',
    title='Order Lifecycle Bottleneck Analysis',
    labels={'Stage': 'Delivery Stage', 'Time (Days)': 'Processing/Shipping Time in Days'},
    points='outliers'
)
fig_boxplot.update_layout(template="plotly_white", title_x=0.5)
fig_boxplot.show()
print("Investigation Complete: The boxplot clearly shows that 'Seller Processing Time' is the primary bottleneck with high variability.")

# ==============================================================================
# END OF ANALYSIS
# ==============================================================================